### Notebook is proof of concept: Algorithm needs refactoring (functional to OO)



### Approach
* expand attendees into independent combos of two
* shuffle and presort list of combos so neighbors don't share elements (this prevents decimation during compression)
* compress combos into independent small groups of specified size
>* must filter list of available combos after each small group formation 
>(eliminate those used in small group formation plus those that are combinations of small group elements in current small group)
* repeat until independent small groups exhausted
* sort small groups so on a per session basis so they are independent 

### Issues
* redundancy threshold parameter must be hand tuned on a case by case basis for best results (see examples below)
* assignment of groups to sessions not yet implemented (note number of sessions is a dependent parameter)
>* a table of possible options concerning small group size and session schedule would be helpful
* due to combo shuffling, results vary slightly with every run
* percentage of dropped 1:1 interactions varies by run but is consistently small when redundancy threshold is tuned correctly, can be fixed by augmenting small groups after analysis but that's klunky 
* need a sorting dashboard to summarize metrics and show performance graphically 

In [1]:
from itertools import combinations
import random
import pandas as pd
import numpy as np
import seaborn as sns
from faker import Faker
fake = Faker()
cm=sns.color_palette("coolwarm", as_cmap=True)

### inputs

In [2]:
large_group_size=14
small_group_size=4
redundancy_thresh=3

### example 1
127 paired interactions 91 required, 21 small groups
* large_group_size=14
* small_group_size=4
* redundancy_thresh=2
### example 2
61 paired interactions, 91 required, 10 small groups
* large_group_size=14
* small_group_size=4
* redundancy_thresh=1
### example 3
69 paired interactions of 91 required, 5 groups of varying size
* large_group_size=14
* small_group_size=7
* redundancy_thresh=2
### example 3
105 paired interactions of 91 required, 5 small groups
* large_group_size=14
* small_group_size=7
* redundancy_thresh=3

In [3]:
names = [fake.unique.first_name() for i in range(large_group_size)]
#names

In [4]:
all_combos=list(combinations(names,2))
#all_combos

In [5]:
random.shuffle(all_combos)
for i in range(len(all_combos)-2):
    j=i+1
    #print(i,j)
    #look ahead for independent combo then swap with neighbor
    while len(set(all_combos[i]+all_combos[j]))<4 and j<len(all_combos)-1:    
        j += 1
    all_combos[i+1], all_combos[j] = all_combos[j], all_combos[i+1]
    #be greedy, look ahead, and try to aggregate 3 independent combos
    j=i+2
    while len(set(all_combos[i]+all_combos[i+1]+all_combos[j]))<6 and j<len(all_combos)-1:    
        j += 1
    all_combos[i+2], all_combos[j] = all_combos[j], all_combos[i+2]
    
len(all_combos)
#all_combos

91

In [6]:
#add flag 'F', used in false_count, set to 'T' when combo is incorporated into a small group
for j in range(len(all_combos)):
    all_combos[j]=all_combos[j][:2]+tuple(['F'])  


In [7]:
small_groups=[] #list of lists containing small groups
false_count=len(all_combos) #remaining unincorporated combos
group_idx=0 

while false_count > 0:
    combo_idx=0 #used when indexing into list of combos
    size_count=0 #tracks progress of adding individuals to small_groups
    next_small_group=[] #list that contains a candidate small group, subject to tests and modifications
  
    #begin building small groups
    while size_count < small_group_size and combo_idx<len(all_combos):
        #check to see if current combo has been used before then append it
        if(all_combos[combo_idx][2]=='F'):
            next_small_group.append(all_combos[combo_idx][0])
            next_small_group=list(set(next_small_group))
            #second individual in a group not added if small group size is odd
            if len(next_small_group)<small_group_size:
                next_small_group.append(all_combos[combo_idx][1])
                next_small_group=list(set(next_small_group))
            #flag combo as used in small group
            all_combos[combo_idx]=all_combos[combo_idx][:2]+tuple(['T'])
            false_count-=1
    
            #if more than 2 attendees of current small group are in a group already constructed 
            #remove those redundant members from current group and reset flag of current combo
            for i in range(len(small_groups)):
                if len(set(small_groups[i]).intersection(set(next_small_group)))>redundancy_thresh:
                    next_small_group=list(set(next_small_group)-set(small_groups[i]).intersection(set(next_small_group)))
                    all_combos[combo_idx]=all_combos[combo_idx][:2]+tuple(['F'])
        
            size_count=len(next_small_group)
 
        combo_idx+=1
    
    #print current small group and intersections with prior small groups
    #print(group_idx, false_count,next_small_group)
    for i in range(len(small_groups)):
        if len(set(small_groups[i]).intersection(set(next_small_group)))>redundancy_thresh:
            print('intersection between',small_groups[i],next_small_group)
    small_groups.append(next_small_group)
    group_idx+=1
  
  
    scrub_idx=0
    # filter all remaining combos contained within current small group   
    false_count=0
    while scrub_idx<len(all_combos):
        if (all_combos[scrub_idx][2]=='F'):
            if len(set(next_small_group).intersection(set(all_combos[scrub_idx])))>1:
                all_combos[scrub_idx]=all_combos[scrub_idx][:2]+tuple(['T']) 
            else:
                false_count+=1
        scrub_idx+=1
        #print(scrub_idx)
  


### now evaluate interactions

In [8]:
#all_combos

In [9]:
df=pd.DataFrame({"Name":names})
n=0
for name in names:
  col=pd.Series(np.zeros(len(names)).T)  
  for i in range (n,len(names)):
    col[i]=0
  df = pd.concat([df, col.to_frame()],axis=1)
  df.columns.values[n+1] = name
  n=n+1

    
for combo_idx in range(len(all_combos)):
  for group_idx in range(len(small_groups)):
    if len(set(all_combos[combo_idx]).intersection(set(small_groups[group_idx])))>1:
      df.at[names.index(all_combos[combo_idx][0]),all_combos[combo_idx][1]]=group_idx+1

index = df.index
index.name = "Grouped Combos"
       
df=df.replace(0,np.NaN)
df.style.background_gradient(cmap=cm,vmin=0,vmax=len(small_groups)).highlight_null('black')






,Name,Jennifer,Melissa,Sara,Donald,Kimberly,Christopher,Zachary,Philip,Brent,Grant,Sally,Tara,Matthew,Robert
Grouped Combos,,,,,,,,,,,,,,,
0,Jennifer,nan,7.000000,11.000000,13.000000,21.000000,18.000000,7.000000,21.000000,18.000000,18.000000,13.000000,4.000000,6.000000,21.000000
1,Melissa,nan,nan,5.000000,14.000000,2.000000,9.000000,9.000000,7.000000,9.000000,2.000000,5.000000,nan,14.000000,14.000000
2,Sara,nan,nan,nan,17.000000,20.000000,20.000000,10.000000,20.000000,5.000000,17.000000,10.000000,4.000000,10.000000,17.000000
3,Donald,nan,nan,nan,nan,22.000000,22.000000,3.000000,11.000000,13.000000,17.000000,22.000000,3.000000,14.000000,17.000000
4,Kimberly,nan,nan,nan,nan,nan,22.000000,8.000000,21.000000,16.000000,16.000000,22.000000,16.000000,8.000000,21.000000
5,Christopher,nan,nan,nan,nan,nan,nan,19.000000,20.000000,18.000000,19.000000,22.000000,12.000000,12.000000,19.000000
6,Zachary,nan,nan,nan,nan,nan,nan,nan,7.000000,9.000000,19.000000,10.000000,3.000000,10.000000,19.000000
7,Philip,nan,nan,nan,nan,nan,nan,nan,nan,nan,15.000000,15.000000,12.000000,12.000000,21.000000
8,Brent,nan,nan,nan,nan,nan,nan,nan,nan,nan,18.000000,13.000000,16.000000,nan,1.000000


In [10]:
interactions=0
for combo_idx in range(len(all_combos)):
  for group_idx in range(len(small_groups)):
  
      #print('combo_idx,size_count,group_idx',combo_idx,size_count,group_idx)
      if len(set(all_combos[combo_idx]).intersection(set(small_groups[group_idx])))>1:
        interactions+=1
        print('interaction',interactions,"combo: ",combo_idx,"present in small_group: ",group_idx)
print('interactions',interactions,':',len(all_combos))



interaction 1 combo:  0 present in small_group:  0
interaction 2 combo:  1 present in small_group:  0
interaction 3 combo:  2 present in small_group:  1
interaction 4 combo:  2 present in small_group:  5
interaction 5 combo:  3 present in small_group:  1
interaction 6 combo:  4 present in small_group:  2
interaction 7 combo:  5 present in small_group:  2
interaction 8 combo:  5 present in small_group:  21
interaction 9 combo:  6 present in small_group:  3
interaction 10 combo:  6 present in small_group:  16
interaction 11 combo:  7 present in small_group:  3
interaction 12 combo:  8 present in small_group:  4
interaction 13 combo:  9 present in small_group:  4
interaction 14 combo:  10 present in small_group:  5
interaction 15 combo:  10 present in small_group:  16
interaction 16 combo:  10 present in small_group:  18
interaction 17 combo:  11 present in small_group:  5
interaction 18 combo:  12 present in small_group:  6
interaction 19 combo:  13 present in small_group:  0
interaction

In [11]:
small_groups

[['Robert', 'Brent', 'Tara', 'Sally'],
 ['Matthew', 'Kimberly', 'Grant', 'Melissa'],
 ['Zachary', 'Donald', 'Tara', 'Christopher'],
 ['Tara', 'Jennifer', 'Grant', 'Sara'],
 ['Sara', 'Brent', 'Melissa', 'Sally'],
 ['Robert', 'Matthew', 'Jennifer', 'Grant'],
 ['Zachary', 'Jennifer', 'Philip', 'Melissa'],
 ['Robert', 'Matthew', 'Zachary', 'Kimberly'],
 ['Zachary', 'Brent', 'Melissa', 'Christopher'],
 ['Matthew', 'Zachary', 'Sara', 'Sally'],
 ['Philip', 'Jennifer', 'Donald', 'Sara'],
 ['Matthew', 'Philip', 'Christopher', 'Tara'],
 ['Jennifer', 'Donald', 'Brent', 'Sally'],
 ['Matthew', 'Robert', 'Donald', 'Melissa'],
 ['Philip', 'Kimberly', 'Grant', 'Sally'],
 ['Grant', 'Kimberly', 'Brent', 'Tara'],
 ['Robert', 'Sara', 'Donald', 'Grant'],
 ['Jennifer', 'Brent', 'Grant', 'Christopher'],
 ['Robert', 'Zachary', 'Grant', 'Christopher'],
 ['Philip', 'Sara', 'Kimberly', 'Christopher'],
 ['Robert', 'Jennifer', 'Philip', 'Kimberly'],
 ['Christopher', 'Donald', 'Kimberly', 'Sally']]